In [ ]:
!pip install --upgrade gensim
# !pip install sentence-transformers -q

In [ ]:
import os
import numpy as np
import csv
import string
import pandas as pd
import nltk
from collections import Counter

import warnings
warnings.filterwarnings('ignore')
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from imblearn.over_sampling import SMOTE

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import StratifiedKFold
from sklearn.base import clone
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import RandomForestClassifier

# from sentence_transformers import SentenceTransformer

import tqdm
import copy
import pickle

import torch
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset

from typing import Tuple

np.random.seed(113)

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
path_ukara = "/content/drive/My Drive/Research/AES Backtranslate/"
path_raw = path_ukara + '/Raw'

path_train_A = path_raw + "/Data A/train_A.csv"
path_dev_A = path_raw + "/Data A/val_A.csv"
path_test_A = path_raw + "/Data A/test_A.csv"

# path_train_A = path_ukara + "/Data A BT 25/train_A_btranslated.csv"
# path_dev_A  = path_ukara + "/Data A BT 25/dev_A_btranslated.csv"
# path_test_A = path_ukara + "/Data A BT 25/test_A_btranslated.csv"

path_train_B = path_raw + "/Data B/train_B.csv"
path_dev_B = path_raw + "/Data B/val_B.csv"
path_test_B = path_raw + "/Data B/test_B.csv"

# path_train_B = path_ukara + "/Data B BT 25/train_B_btranslated.csv"
# path_dev_B  = path_ukara + "/Data B BT 25/dev_B_btranslated.csv"
# path_test_B = path_ukara + "/Data B BT 25/data_test_B_btranslated.csv"

path_train_A_bt = path_raw + '/Data A BT/data_train_A_combined.csv'
path_train_B_bt = path_raw + '/Data B BT/data_train_B_combined.csv'

# path_spellcheck = path_ukara + '/spellcheck.txt'
path_spellcheck = 'spellcheck.txt'

In [ ]:
train_map = {
  "A": path_train_A,
  "B": path_train_B,
  "A-bt": path_train_A_bt,
}

valid_map = {
  "A": path_dev_A,
  "B": path_dev_B,
}

test_map = {
  "A": path_test_A,
  "B": path_test_B,
}

dictSoal = {}
for key, value in train_map.items():
  dictSoal[key] = pd.read_csv(value)

dictDev = {}
for key, value in valid_map.items():
  dictDev[key] = pd.read_csv(value)

dictTest = {}
for key, value in test_map.items():
  dictTest[key] = pd.read_csv(value)

In [ ]:
# Drop rows with empty responses
for dicts in [dictSoal, dictDev, dictTest]:
  for key in dicts:
    dicts[key] = dicts[key].dropna(subset=["RESPONSE"], axis=0)

# Preprocessing

### Abbreviation Normalization

In [ ]:
df = pd.read_csv(path_ukara + '/dictNorm.csv', index_col=0)

df = df.where(pd.notnull(df), ' ')
dictNorm = df.to_dict()["after"]

def norm_text(text, printer=False):
  if printer:
    print(dictNorm)
  for wBefore, wAfter in dictNorm.items():
      text = wAfter if text == wBefore else text
  text = text.replace('\n', ' ').replace('\\', ' ')
  return text

### Misspelling correction

In [ ]:
from collections import Counter
import re

CHARACTERS = ['.',',',';',':','-,','...','?','!','(',')','[',']','{','}','<','>','"','/','\'','#','-','@']

class MisspellingCorrector():
  def __init__(self, path_spellcheck):
    self.path_spellcheck = path_spellcheck
    self.WORDS = Counter(self._words(open(path_spellcheck).read()))

  def _words(self, text): return re.findall(r'\w+', text.lower())


  def _P(self, word, N = None):
      N = N or sum(self.WORDS.values())
      # "Probability of `word`."
      return self.WORDS[word] / N

  def _correct_word(self, word):
      # "Most probable spelling correction for word."
      return max(self._candidates(word), key=self._P)

  def correct_text(self, text):
      spellCheck = []
      for i in text:
        if i not in CHARACTERS:
          j = self._correct_word(i)
          spellCheck.append(j)
        else:
          spellCheck.append(i)

      return spellCheck

  def _candidates(self, word):
      # "Generate possible spelling corrections for word."
      return (self._known([word]) or self._known(self._edits1(word)) or self._known(self._edits2(word)) or [word])

  def _known(self, words):
      # "The subset of `words` that appear in the dictionary of WORDS."
      return set(w for w in words if w in self.WORDS)

  def _edits1(self, word):
      # "All edits that are one edit away from `word`."
      letters    = 'abcdefghijklmnopqrstuvwxyz'
      splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)] # [('', 'kemarin'), ('k', 'emarin'), ('ke', 'marin'), dst]
      deletes    = [L + R[1:]               for L, R in splits if R] # ['emarin', 'kmarin', 'kearin', dst]
      transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1] # ['ekmarin', 'kmearin', 'keamrin', dst]
      replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters] # ['aemarin', 'bemarin', 'cemarin', dst]
      inserts    = [L + c + R               for L, R in splits for c in letters] # ['akemarin', 'bkemarin', 'ckemarin', dst]
      return set(deletes + transposes + replaces + inserts)

  def _edits2(self, word):
      # "All edits that are two edits away from `word`."
      return (e2 for e1 in self._edits1(word) for e2 in self._edits1(e1))


In [ ]:
import re
import string

class Tokenizer():
  def tokenize_word(self, sentence, splitby='space'):
    if splitby.strip().lower()=='space':
      words = re.split(r'\s',sentence)
    elif splitby.strip().lower()=='word':
      words = re.split(r'(\w+)?', sentence)
    else:
      raise NotImplementedError

    words = [x.strip().lower() for x in words if x.strip()]

    return words

  def tokenize_char(self,word):
    return list(word)



## Run Dataset Transform

In [ ]:
def correct_text_map(text, tok: Tokenizer, corrector: MisspellingCorrector, printer=False):
  print(text)
  text = text.encode("utf-8")
  text = str(text.decode("utf-8"))

  if printer:
    print('before', text)
  words = tok.tokenize_word(text)
  normalized_words = [norm_text(w) for w in words]
  corrected_words = corrector.correct_text(normalized_words)
  text = ' '.join(corrected_words)
  if printer:
    print('after ', text)

  return text

def transform(prob):
  tok = Tokenizer()
  corrector = MisspellingCorrector(path_spellcheck)
  map_func = np.vectorize(correct_text_map)

  # Get raw datasets
  X_train = dictSoal[prob].RESPONSE
  Y_train = dictSoal[prob].LABEL.to_numpy()
  X_val = dictDev[prob].RESPONSE
  Y_val = dictDev[prob].LABEL.to_numpy()
  X_test = dictTest[prob].RESPONSE
  Y_test = dictTest[prob].LABEL.to_numpy()

  # Get embeddings
  pp_test_x = map_func(X_test, tok, corrector)
  pp_train_x = map_func(X_train, tok, corrector)
  pp_val_x = map_func(X_val, tok, corrector)

  pp_test = pd.DataFrame({'RESPONSE': pp_test_x, 'LABEL': Y_test})
  pp_train = pd.DataFrame({'RESPONSE': pp_train_x, 'LABEL': Y_train})
  pp_val = pd.DataFrame({'RESPONSE': pp_val_x, 'LABEL': Y_val})

  return pp_train, pp_val, pp_test



## Save Dataset

In [ ]:
# output_path = path_ukara + '/SavedDatasets/'
# def save(df: pd.DataFrame, name: str, output_path: str = path_ukara + '/SavedDatasets/'):
#  pd.to_csv(output_path + name)

# save(df, "pp-misspell-norm-1")

In [ ]:
# !cp dumped.pkl {output_path + name}

In [ ]:
pp_train, pp_val, pp_test = transform("A")
pp_train.to_csv('train_A_pp.csv')
pp_test.to_csv('test_A_pp.csv')
pp_val.to_csv('val_A_pp.csv')

beradaptasi dengan lingkunagn baru, lahan dan harta mereka yang dulu akan hilang, tertinggalnya teknologi karena bencana lingkunagn tersebut.
beradaptasi dengan lingkunagn baru, lahan dan harta mereka yang dulu akan hilang, tertinggalnya teknologi karena bencana lingkunagn tersebut.
Pengungsi akan mencari mata pencaharian yang baru karena mereka kehilangan lahan pertanian / matapencaharian
komunitas dan negaranya.
mereka perlu menyesuaikan diri dan beradaptasi dengan lingkungan yang baru.
TANTANGNNYA ADALAH KEBUTUHAN EKONOMI
Tantangan yang harus dihadapi adalah jika warga Travina harus berpindah tempat, mereka harus beradaptasi dengan tempat tinggal baru mereka mulai dari lahan pertanian untuk pekerjaan,bahan pangan dan lainnya.
akan kesulitan mendapatkan lahan dan tempat tinggal
susah beradaptasi... harus dapat menyesuaikan lingkungan nya
salah satu yang terjadi adalah.....
mengurangi dampak yang menyebabkan pemanasan global
lahan lahan pertanian,rumah mereka terpaksa hilang karena ha

In [ ]:
# output_path = path_ukara + '/SavedDatasets' + '/pp-misspell-norm-1/'
# pp_train.to_csv('./' + 'train_A.csv')
# pp_test.to_csv('./' + 'test_A.csv')
# pp_val.to_csv('./' + 'val_A.csv')

In [ ]:
pp_train, pp_val, pp_test = transform("B")
# pp_val = transform("B")
# pp_train.to_csv('./' + 'train_B.csv')
# pp_test.to_csv('./' + 'test_B.csv')
# pp_val.to_csv('./' + 'val_B.csv')

pp_train.to_csv('train_B_pp2.csv')
pp_test.to_csv('test_B_pp2.csv')
pp_val.to_csv('val_B_pp2.csv')

menyumbang pakaian
menyumbang pakaian
karena kebanyakan orang menyumbangkan 2 euro tersebut sebagai upaya produksi pakaian menjadi lebih beretika. karena pasti kebanyakan orang menginginkan produksi pakaian menjadi lebih beretika.
karena barang yang disumbang akan lebih bermanfaat kepada mereka yang membutuhkan
karena mereka udah ditampilkan gambar tersebut. mereka punya rasa empati dan kasihan.
karena sebagai upaya untuk membuat produksi pakaian menjadi lebih ber etika.
karena pakaian yang mereka pakai sudah tidak cukup dan sayang jika pakaiannya dibuang.lebih baik disumbangkan
karena menyumbang itu adalah tindakan yang sangat terpuji
karena memilih jalan aman setelah melihat kondidi kerja di pabrik
karena menyumbang akan mengurangi dampak sampah lingkungan.
supaya mendapatksan pahala yang besar
karena menyumbang sebagian dari iman dan memudah kan kitah untuk sokses
karena Setelahnya, konsumen diberi pilihan apakah akan melanjutkan pembelian atau menyumbangkan 2 Euro tersebut sebagai 

In [ ]:
# pp_train.to_csv('./' + 'train_B.csv')
# pp_test.to_csv('./' + 'test_B.csv')
# pp_val.to_csv('./' + 'val_B.csv')